# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [4]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-sport2-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Running explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sports. Compare awards and participation in the competitions of the players

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`       | Human         | node      |
| `wd:Q9124`    | Eliud Kipchoge| node      |
| `wd:Q853003`| athletics at the 2008 Summer Olympics – men's 200 metres |node|





Also consider

```
wd:Q9124 ?p ?obj .
```

is the BGP to retrieve all **properties of Eliud Kipchoge**


Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for runner

2. Identify the BGP for long distance running

3. Return the disciplines of the long distance running (e.g. marathon) (the result set must be a list of couples discipline IRI and label).

4. Consider marathons ran in Europe. Return the number of different runners who won a marathon group by their country of citizenship (the result set must be a triples of country IRI, label and #different runners).

5. Identify the BGP for Olympic Games

6. Consider only the summer olympic games of 2004-2008-2012-2016. Return the number of marathon runners who had partecipated to all these editions group by their country of citizenship (if the runner partecipated in more than one edition of the game, consider him/her only once) (the result set must be a list of country IRI, label and #marathon runners)

7. For each disciplines of the long distance running find who hold the World Record (the result set must be a list of elements with discipline IRI, label and athlete IRI and label).

In [7]:
## startup the evaluation
# setup the file and create the empty json
ipname = "s2.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 0_2


## Task 1 
Identify the BGP for runner

In [8]:
# starting from Eliud Kipchoge I look at his properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q9124 ?pr ?obj.
    ?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('pr', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('pr', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('pr', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('pr', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('pr', 'http://www.wikidata.org/prop/direct/P1741'), ('name', 'GTAA ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('pr', 'http://www.wikidata.org/prop/direct/P19'

In [9]:
# starting from Eliud Kipchoge I look at his occupations
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q9126 wdt:P106 ?obj.
    ?obj sc:name ?name .
} 
LIMIT 100
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11513337'), ('name', 'athletics competitor')]
[('obj', 'http://www.wikidata.org/entity/Q4439155'), ('name', 'long-distance runner')]
2


In [10]:
# look at super class of the occupations found above
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q9126 wdt:P106/wdt:P279* ?obj.
    ?obj sc:name ?name .
} 
LIMIT 100
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11513337'), ('name', 'athletics competitor')]
[('obj', 'http://www.wikidata.org/entity/Q4439155'), ('name', 'long-distance runner')]
[('obj', 'http://www.wikidata.org/entity/Q2066131'), ('name', 'athlete')]
[('obj', 'http://www.wikidata.org/entity/Q103940464'), ('name', 'continuant')]
[('obj', 'http://www.wikidata.org/entity/Q215627'), ('name', 'person')]
[('obj', 'http://www.wikidata.org/entity/Q35120'), ('name', 'entity')]
[('obj', 'http://www.wikidata.org/entity/Q12803959'), ('name', 'runner')]
[('obj', 'http://www.wikidata.org/entity/Q23958946'), ('name', 'individual entity')]
[('obj', 'http://www.wikidata.org/entity/Q795052'), ('name', 'individual')]
[('obj', 'http://www.wikidata.org/entity/Q50995749'), ('name', 'sportsperson')]
[('obj', 'http://www.wikidata.org/entity/Q830077'), ('name', 'subject')]
[('obj', 'http://www.wikidata.org/entity/Q24229398'), ('name', 'agent')]
[('obj', 'http://www.wikidata.org/entity/Q18336849'), ('name

In [11]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q12803959"
og_name = "runner"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_2
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_2.json
JSON object updated


`wd:Q12803959` is the runner in wikidata

## Task 2
Identify the BGP for long distance running

In [12]:
# in the first query I found Q12803959 that is the class of runner. Investigate the properties
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    ?people wdt:P106/wdt:P279* wd:Q12803959 ;
            ?prop ?x.
    ?prop sc:name ?name .
    FILTER(REGEX(?name,"[S,s]port")).
    FILTER(!isLiteral(?x)).
}
"""

print("Results")
x= run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('prop', 'http://www.wikidata.org/prop/direct/P2416'), ('name', 'sports discipline competed in')]
[('prop', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]
[('prop', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('prop', 'http://www.wikidata.org/prop/direct/P6758'), ('name', 'supported sports team')]
[('prop', 'http://www.wikidata.org/prop/direct/P6087'), ('name', 'coach of sports team')]
[('prop', 'http://www.wikidata.org/prop/direct/P1327'), ('name', 'partner in business or sport')]
[('prop', 'http://www.wikidata.org/prop/direct/P5249'), ('name', 'sports competition competed at')]
8


In [13]:
# using the property sport (P641) I print some results
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    ?people wdt:P106/wdt:P279* wd:Q12803959 ;
            wdt:P641 ?x.
    ?x sc:name ?name .
    FILTER(REGEX(?name,"unnin")).
}
"""

print("Results")
x= run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q500050'), ('name', 'cross country running')]
[('x', 'http://www.wikidata.org/entity/Q1163256'), ('name', 'middle-distance running')]
[('x', 'http://www.wikidata.org/entity/Q917206'), ('name', 'long-distance running')]
[('x', 'http://www.wikidata.org/entity/Q1934817'), ('name', 'road running')]
[('x', 'http://www.wikidata.org/entity/Q2167372'), ('name', 'mountain running')]
[('x', 'http://www.wikidata.org/entity/Q2093999'), ('name', 'trail running')]
[('x', 'http://www.wikidata.org/entity/Q105674'), ('name', 'running')]
[('x', 'http://www.wikidata.org/entity/Q3962667'), ('name', 'skyrunning')]
8


In [14]:
### insert the result of TASK 2 in the file
og_uri = "http://www.wikidata.org/entity/Q917206"
og_name = "long distance running"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_2
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_2.json
JSON object updated


`wd:Q917206` is the long distance running

## Task 3
Identify all the disciplines of the long distance running (e.g. 10000 metres, marathon)

In [15]:
# print the entities that are instance of long distance running
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    ?x wdt:P31 wd:Q917206 .
    ?x sc:name ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q15140178'), ('name', '10 et 20 km de Tours')]
[('x', 'http://www.wikidata.org/entity/Q3512341'), ('name', 'Terry Fox Run')]
[('x', 'http://www.wikidata.org/entity/Q1392915'), ('name', 'Tatry Running Tour')]
[('x', 'http://www.wikidata.org/entity/Q2806693'), ('name', '100 km du Morvan')]
[('x', 'http://www.wikidata.org/entity/Q19843758'), ('name', 'Okpekpe 10km Road Race')]
[('x', 'http://www.wikidata.org/entity/Q57016051'), ('name', 'Taubertal 100')]
6


In [16]:
# poor results, explore the long distance running
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q917206 ?prop ?x.
    ?prop sc:name ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P3095'), ('name', 'practiced by')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P3553'), ('name', 'Zhihu topic ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Common

In [18]:
# use the property has part (P527)
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q917206 wdt:P527 ?x.
    ?x sc:name ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q163892'), ('name', '10,000 metres')]
[('x', 'http://www.wikidata.org/entity/Q240500'), ('name', '5000 metres')]
[('x', 'http://www.wikidata.org/entity/Q40244'), ('name', 'marathon')]
[('x', 'http://www.wikidata.org/entity/Q500050'), ('name', 'cross country running')]
[('x', 'http://www.wikidata.org/entity/Q215677'), ('name', 'half marathon')]
[('x', 'http://www.wikidata.org/entity/Q9171043'), ('name', '5K run')]
[('x', 'http://www.wikidata.org/entity/Q1202167'), ('name', 'ekiden')]
[('x', 'http://www.wikidata.org/entity/Q2093999'), ('name', 'trail running')]
[('x', 'http://www.wikidata.org/entity/Q2774730'), ('name', '10K run')]
9


In [19]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 0_2
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_2.json
JSON object updated


## Task 4
Consider marathons ran in Europe. For each country (world wide) find how many runners won a marathon

First of all I need to find the wikidata code for Europe

In [20]:
# find the nationality of Kipchoge
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q9124 wdt:P27 ?obj.
    ?obj sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q114'), ('name', 'Kenya')]
1


In [21]:
# find the instance of country
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    wd:Q9124 wdt:P27 ?obj.
    ?obj wdt:P31 ?inst.
    ?inst sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('inst', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
2


In [22]:
# look for the property continent
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
    ?obj wdt:P31 wd:Q6256;
         ?prop ?x.
    ?prop sc:name ?name .
    FILTER(REGEX(?name,"ontinen")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
1


In [23]:
# print the continents
queryString = """
SELECT DISTINCT ?x ?name
WHERE {
    ?obj wdt:P31 wd:Q6256;
         wdt:P30 ?x.
    ?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('x', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America')]
[('x', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('x', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('x', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('x', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
[('x', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia')]
[('x', 'http://www.wikidata.org/entity/Q27611'), ('name', 'Central America')]
8


Recap:
- `wd:Q40244`: marathon
- `wd:Q46`: Europe
- `wd:Q6256`: Country

In [24]:
# check if there exists something that relate country in Europe with instances of marathon 
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
    ?country wdt:P31 wd:Q6256;
         wdt:P30 wd:Q46.
    ?marathon wdt:P31 wd:Q40244;
        ?prop ?country.
    ?prop sc:name ?name .
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
1


In [25]:
# print some example of couple marathon - country
queryString = """
SELECT DISTINCT ?marathon ?name ?country ?cName
WHERE {
    ?country wdt:P31 wd:Q6256;
         wdt:P30 wd:Q46.
    ?marathon wdt:P31 wd:Q40244;
        wdt:P17 ?country.
    ?marathon sc:name ?name .
    ?country sc:name ?cName .
}
LIMIT 20
"""

print("Results")
x = run_query(queryString)

Results
[('marathon', 'http://www.wikidata.org/entity/Q1645007'), ('name', 'Treviso Marathon'), ('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy')]
[('marathon', 'http://www.wikidata.org/entity/Q1071285'), ('name', 'Paris Marathon'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cName', 'France')]
[('marathon', 'http://www.wikidata.org/entity/Q2029795'), ('name', "1987 World Championships in Athletics – Men's Marathon"), ('country', 'http://www.wikidata.org/entity/Q38'), ('cName', 'Italy')]
[('marathon', 'http://www.wikidata.org/entity/Q2031637'), ('name', 'Prague Marathon'), ('country', 'http://www.wikidata.org/entity/Q213'), ('cName', 'Czech Republic')]
[('marathon', 'http://www.wikidata.org/entity/Q3702630'), ('name', "1995 World Championships in Athletics – men's marathon"), ('country', 'http://www.wikidata.org/entity/Q34'), ('cName', 'Sweden')]
[('marathon', 'http://www.wikidata.org/entity/Q4982646'), ('name', 'Bucharest Marathon'), ('country', 'http://w

In [26]:
# find the winner of the marathon. Print marathons' properties
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
    ?marathon wdt:P31 wd:Q40244;
              ?prop ?runner.
    ?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P10'), ('name', 'video')]
[('prop', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P1448'), ('name', 'official name')]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/p

In [27]:
# use the property winner (P1346)
queryString = """
SELECT DISTINCT ?runner ?name
WHERE {
    ?marathon wdt:P31 wd:Q40244;
              wdt:P1346 ?runner.
    ?runner sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('runner', 'http://www.wikidata.org/entity/Q1661572'), ('name', 'Leonid Moseyev')]
[('runner', 'http://www.wikidata.org/entity/Q357206'), ('name', 'Ian Thompson')]
[('runner', 'http://www.wikidata.org/entity/Q1359466'), ('name', 'Jack Holden')]
[('runner', 'http://www.wikidata.org/entity/Q180891'), ('name', 'Sergei Popov')]
[('runner', 'http://www.wikidata.org/entity/Q912468'), ('name', 'Brian Kilby')]
[('runner', 'http://www.wikidata.org/entity/Q558268'), ('name', 'Karel Lismont')]
[('runner', 'http://www.wikidata.org/entity/Q246136'), ('name', 'Veikko Karvonen')]
[('runner', 'http://www.wikidata.org/entity/Q1689117'), ('name', 'Jim Hogan')]
[('runner', 'http://www.wikidata.org/entity/Q356579'), ('name', 'Ron Hill')]
[('runner', 'http://www.wikidata.org/entity/Q217614'), ('name', 'Armas Toivonen')]
[('runner', 'http://www.wikidata.org/entity/Q1343015'), ('name', 'Väinö Muinonen')]
[('runner', 'http://www.wikidata.org/entity/Q354905'), ('name', 'Mikko Hietanen')]
12


Poor result probably because I have to consider editions of marathons. Explore the marathons editions

In [28]:
# check the instance of
queryString = """
SELECT DISTINCT ?edition ?name
WHERE {
    ?marathon wdt:P31 wd:Q40244.
    ?edition wdt:P31 ?marathon.
    ?edition sc:name ?name .
}
LIMIT 50
"""

print("Results")
x = run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q96370343'), ('name', '2020 London Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q1817967'), ('name', '2006 New York City Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q1918370'), ('name', '2007 New York City Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q1927506'), ('name', '1999 Paris Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q1952434'), ('name', '2000 Paris Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q2689716'), ('name', '1998 Paris Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q3028553'), ('name', '2008 New York City Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q5315869'), ('name', '2007 Frankfurt Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q2062694'), ('name', '2003 Rotterdam Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q2330438'), ('name', '2002 Rotterdam Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q2447905'

In [29]:
# check the HAS PART
queryString = """
SELECT DISTINCT ?edition ?name
WHERE {
    ?marathon wdt:P31 wd:Q40244.
    ?marathon wdt:P527 ?edition .
    ?edition sc:name ?name .
}
LIMIT 50
"""

print("Results")
x = run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q17496398'), ('name', "2014 European Athletics Championships – Women's Marathon")]
[('edition', 'http://www.wikidata.org/entity/Q57933742'), ('name', '2018 Frankfurt Marathon')]
[('edition', 'http://www.wikidata.org/entity/Q17573153'), ('name', "2014 European Athletics Championships – Men's Marathon")]
3


In [30]:
# check the subclasses
queryString = """
SELECT DISTINCT ?edition ?name
WHERE {
    ?marathon wdt:P31 wd:Q40244.
    ?edition wdt:P279 ?marathon .
    ?edition sc:name ?name .
}
LIMIT 50
"""

print("Results")
x = run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q83472812'), ('name', 'Köln Marathon 2014')]
[('edition', 'http://www.wikidata.org/entity/Q83473125'), ('name', 'Köln Marathon 2015')]
[('edition', 'http://www.wikidata.org/entity/Q83472951'), ('name', 'Köln Marathon 2016')]
3


In [31]:
# check the properties of the editions
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
    ?marathon wdt:P31 wd:Q40244.
    ?edition wdt:P31 ?marathon.
    ?edition ?prop ?x.
    ?prop sc:name ?name .
}
LIMIT 50
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1545'), ('name', 'series ordinal')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P2416'), ('name', 'sports discipline competed in')]
[('prop', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop

In [32]:
# return some marathon edition with its winner
queryString = """
SELECT DISTINCT ?edition ?eName ?runner ?rName
WHERE {
    ?country wdt:P31 wd:Q6256;
         wdt:P30 wd:Q46.
    ?marathon wdt:P279*/wdt:P31 wd:Q40244;
        wdt:P17 ?country.
    ?edition wdt:P31 ?marathon.
    ?runner wdt:P106/wdt:P279* wd:Q12803959.
    ?edition wdt:P1346 ?runner.
    ?edition sc:name ?eName .
    ?runner sc:name ?rName .
}
LIMIT 50
"""

print("Results")
x = run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q16802238'), ('eName', '2014 Paris Marathon'), ('runner', 'http://www.wikidata.org/entity/Q9119'), ('rName', 'Kenenisa Bekele Beyecha')]
[('edition', 'http://www.wikidata.org/entity/Q13813997'), ('eName', '1996 Paris Marathon'), ('runner', 'http://www.wikidata.org/entity/Q16337559'), ('rName', 'Henrique Crisóstomo')]
[('edition', 'http://www.wikidata.org/entity/Q29053996'), ('eName', '2017 Paris Marathon'), ('runner', 'http://www.wikidata.org/entity/Q11831849'), ('rName', 'Purity Cherotich Rionoripo')]
[('edition', 'http://www.wikidata.org/entity/Q29053996'), ('eName', '2017 Paris Marathon'), ('runner', 'http://www.wikidata.org/entity/Q21414453'), ('rName', 'Paul Kipchumba Lonyangata')]
[('edition', 'http://www.wikidata.org/entity/Q51687336'), ('eName', '2018 Paris Marathon'), ('runner', 'http://www.wikidata.org/entity/Q21414453'), ('rName', 'Paul Kipchumba Lonyangata')]
[('edition', 'http://www.wikidata.org/entity/Q65234870'), ('eNa

In [44]:
# group the results by country
queryString = """
SELECT ?nation ?name COUNT(DISTINCT ?runner) AS ?winners
WHERE {
    ?country wdt:P31 wd:Q6256;
             wdt:P30 wd:Q46.
    ?marathon wdt:P31 wd:Q40244;
        wdt:P17 ?country.
    ?edition wdt:P31 ?marathon.
    ?runner wdt:P106/wdt:P279* wd:Q12803959.
    ?edition wdt:P1346 ?runner.
    ?runner wdt:P27 ?nation.
    ?nation sc:name ?name .
}
GROUP BY ?nation ?name
"""

print("Results")
x = run_query(queryString)

Results
[('nation', 'http://www.wikidata.org/entity/Q45'), ('name', 'Portugal'), ('winners', '1')]
[('nation', 'http://www.wikidata.org/entity/Q115'), ('name', 'Ethiopia'), ('winners', '10')]
[('nation', 'http://www.wikidata.org/entity/Q114'), ('name', 'Kenya'), ('winners', '16')]
[('nation', 'http://www.wikidata.org/entity/Q924'), ('name', 'Tanzania'), ('winners', '1')]
[('nation', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan'), ('winners', '1')]
[('nation', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America'), ('winners', '1')]
6


In [45]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    winners = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= winners
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_2
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_2.json
JSON object updated


Even if are not present a lot of editions, it is clear that Kenya and Ethiopia are very good in run the marathon.

## Task 5

In [48]:
# from a given URI look at its properties
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
    ?obj ?prop wd:Q853003 .
    ?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
4


In [50]:
# use the has part property
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    ?obj wdt:P527 wd:Q853003 .
    ?obj sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q189941'), ('name', 'athletics at the 2008 Summer Olympics')]
1


In [51]:
# use the has part property
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
    ?obj wdt:P527 ?x .
    ?x wdt:P527 wd:Q853003 .
    ?obj sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8567'), ('name', '2008 Summer Olympics')]
1


In [53]:
# instance of 2008 Summer Olympics
queryString = """
SELECT DISTINCT ?inst ?name
WHERE {
    ?obj wdt:P527 ?x .
    ?x wdt:P527 wd:Q853003 .
    ?obj wdt:P31 ?inst.
    ?inst sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
1


In [62]:
# properties related to Summer Olympic Games
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
    ?inst ?prop wd:Q159821 .
    ?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('prop', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('prop', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('prop', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P461'), ('name', 'opposite of')]
[('prop', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
[('prop', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
11


In [63]:
# find Olympic Games
queryString = """
SELECT DISTINCT ?inst ?name
WHERE {
    ?inst wdt:P2670 wd:Q159821 .
    ?inst sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]
1


In [65]:
### insert the result of TASK 6 in the file
og_uri = "http://www.wikidata.org/entity/Q5389"
og_name = "Olympic Games"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_2
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_2.json
JSON object updated


## Task 6

In [67]:
# find the four Olympic Games of interest
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    ?inst wdt:P31 wd:Q159821.
    ?inst sc:name ?name .
}
ORDER BY ?name
"""

print("Results")
x = run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q8080'), ('name', '1896 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8105'), ('name', '1906 Intercalated Games')]
[('inst', 'http://www.wikidata.org/entity/Q8111'), ('name', '1908 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8118'), ('name', '1912 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8123'), ('name', '1916 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8128'), ('name', '1920 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8132'), ('name', '1924 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8138'), ('name', '1928 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8143'), ('name', '1932 Summer Olympics')]
[('inst', 'http://www.wikidata.org/entity/Q8150'), ('n

In [68]:
# (wd:Q8558,wd:Q8567,wd:Q8577,wd:Q8613) the editions from 2004 to 2016 of the Summer Olympics
queryString = """
SELECT DISTINCT ?edition ?name
WHERE { 
    VALUES ?edition {wd:Q8558 wd:Q8567 wd:Q8577 wd:Q8613}.
    ?edition sc:name ?name .
}
ORDER BY ?name
"""

print("Results")
x = run_query(queryString)

Results
[('edition', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8567'), ('name', '2008 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
[('edition', 'http://www.wikidata.org/entity/Q8613'), ('name', '2016 Summer Olympics')]
4


In [103]:
# find how the runners are related to the summer olympics
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    VALUES ?edition {wd:Q8558 wd:Q8567 wd:Q8577 wd:Q8613}.
    ?runner wdt:P106 wd:Q13382460.
    ?runner ?prop ?edition.
    ?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
1


In [71]:
# find how the runners are related to the summer olympics
queryString = """
SELECT DISTINCT COUNT(*)
WHERE { 
    VALUES ?edition {wd:Q8558 wd:Q8567 wd:Q8577 wd:Q8613}.
    ?runner wdt:P106 wd:Q13382460.
    ?runner ?prop ?edition.
    ?prop sc:name ?name .
}
ORDER BY ?name
"""

print("Results")
x = run_query(queryString)

Results
[('callret-0', '505')]
1


In [92]:
# try a different approach. Find the marathon at the Olympic Games and check runners who had partecipated at the marathons
# from the athletics at the 2008 Summer Olympics
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q189941 ?prop ?x.
    ?prop sc:name ?name .
    FILTER(!isLiteral(?x)).
}
ORDER BY ?name
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('prop', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
11


In [93]:
# try a different approach. Find the marathon at the Olympic Games and check runners who had partecipated at the marathons
# from the athletics at the 2008 Summer Olympics
# use hasPart to get all the disciplines
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q189941 wdt:P527 ?x.
    ?x sc:name ?name .
    FILTER(CONTAINS(?name,"arath")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q693595'), ('name', "athletics at the 2008 Summer Olympics – men's marathon")]
[('x', 'http://www.wikidata.org/entity/Q947691'), ('name', "athletics at the 2008 Summer Olympics – women's marathon")]
2


In [94]:
# try a different approach. Find the marathon at the Olympic Games and check runners who had partecipated at the marathons
# from the athletics at the 2008 Summer Olympics
# use hasPart to get all the disciplines
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
    wd:Q189941 wdt:P527 ?x.
    ?x sc:name ?xn .
    FILTER(CONTAINS(?xn,"arath")).
    
    ?runner wdt:P106 wd:Q13382460.
    ?runner ?prop ?x.
    ?prop sc:name ?name.
    
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
1


In [95]:
# try a different approach. Find the marathon at the Olympic Games and check runners who had partecipated at the marathons
# from the athletics at the 2008 Summer Olympics
# find all the athletics at the Summer Olympics for 2004-2008-2012-2016
queryString = """
SELECT DISTINCT ?x ?xn
WHERE {
    VALUES ?edition {wd:Q8558 wd:Q8567 wd:Q8577 wd:Q8613}.
    ?edition wdt:P527/wdt:P527 ?x.
    ?x sc:name ?xn .
    FILTER(CONTAINS(?xn,"arath")).
    
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q1004931'), ('xn', "swimming at the 2008 Summer Olympics – men's marathon 10 kilometre")]
[('x', 'http://www.wikidata.org/entity/Q739527'), ('xn', "swimming at the 2008 Summer Olympics – women's marathon 10 kilometre")]
[('x', 'http://www.wikidata.org/entity/Q2000988'), ('xn', "athletics at the 2012 Summer Olympics – women's marathon")]
[('x', 'http://www.wikidata.org/entity/Q21858030'), ('xn', "athletics at the 2016 Summer Olympics – women's marathon")]
[('x', 'http://www.wikidata.org/entity/Q1064845'), ('xn', "swimming at the 2012 Summer Olympics – women's marathon 10 kilometre")]
[('x', 'http://www.wikidata.org/entity/Q21858018'), ('xn', "athletics at the 2016 Summer Olympics – men's marathon")]
[('x', 'http://www.wikidata.org/entity/Q25991470'), ('xn', "swimming at the 2016 Summer Olympics – women's marathon 10 kilometre")]
[('x', 'http://www.wikidata.org/entity/Q693595'), ('xn', "athletics at the 2008 Summer Olympics – men's marathon"

In [96]:
# try a different approach. Find the marathon at the Olympic Games and check runners who had partecipated at the marathons
# from the athletics at the 2008 Summer Olympics
# find all the athletics at the Summer Olympics for 2004-2008-2012-2016
# filter only athletics, so remove swimming using property sport
queryString = """
SELECT DISTINCT ?sport ?s
WHERE {
    VALUES ?edition {wd:Q8558 wd:Q8567 wd:Q8577 wd:Q8613}.
    ?edition wdt:P527/wdt:P527 ?x.
    ?x wdt:P641 ?sport.
    ?x sc:name ?xn .
    FILTER(CONTAINS(?xn,"arath")).
    ?sport sc:name ?s.
    
}
"""

print("Results")
x = run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q31920'), ('s', 'swimming')]
[('sport', 'http://www.wikidata.org/entity/Q542'), ('s', 'athletics')]
[('sport', 'http://www.wikidata.org/entity/Q631004'), ('s', 'open water swimming')]
3


In [97]:
# try a different approach. Find the marathon at the Olympic Games and check runners who had partecipated at the marathons
# from the athletics at the 2008 Summer Olympics
# find all the athletics at the Summer Olympics for 2004-2008-2012-2016
# filter only athletics, so remove swimming using property sport
queryString = """
SELECT DISTINCT ?x ?xn
WHERE {
    VALUES ?edition {wd:Q8558 wd:Q8567 wd:Q8577 wd:Q8613}.
    ?edition wdt:P527/wdt:P527 ?x.
    ?x wdt:P641 wd:Q542.
    ?x sc:name ?xn .
    FILTER(CONTAINS(?xn,"arath")).
    
}
ORDER BY ?xn
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q2552272'), ('xn', "athletics at the 2004 Summer Olympics – women's marathon")]
[('x', 'http://www.wikidata.org/entity/Q693595'), ('xn', "athletics at the 2008 Summer Olympics – men's marathon")]
[('x', 'http://www.wikidata.org/entity/Q947691'), ('xn', "athletics at the 2008 Summer Olympics – women's marathon")]
[('x', 'http://www.wikidata.org/entity/Q2443986'), ('xn', "athletics at the 2004 Summer Olympics – men's marathon")]
[('x', 'http://www.wikidata.org/entity/Q21858030'), ('xn', "athletics at the 2016 Summer Olympics – women's marathon")]
[('x', 'http://www.wikidata.org/entity/Q2000988'), ('xn', "athletics at the 2012 Summer Olympics – women's marathon")]
[('x', 'http://www.wikidata.org/entity/Q21858018'), ('xn', "athletics at the 2016 Summer Olympics – men's marathon")]
[('x', 'http://www.wikidata.org/entity/Q1798983'), ('xn', "athletics at the 2012 Summer Olympics – men's marathon")]
8


In [98]:
# try a different approach. Find the marathon at the Olympic Games and check runners who had partecipated at the marathons
# from the athletics at the 2008 Summer Olympics
# find all the athletics at the Summer Olympics for 2004-2008-2012-2016
# filter only athletics, so remove swimming using property sport
queryString = """
SELECT DISTINCT COUNT(*)
WHERE {
    VALUES ?edition {wd:Q8558 wd:Q8567 wd:Q8577 wd:Q8613}.
    ?edition wdt:P527/wdt:P527 ?x.
    ?x wdt:P641 wd:Q542.
    ?runner wdt:P1344 ?x.
    ?x sc:name ?xn .
    FILTER(CONTAINS(?xn,"arath")).
    
}
"""

print("Results")
x = run_query(queryString)

Results
[('callret-0', '61')]
1


Given that using the second strategy the results are lower than using the first strategy, we keep the first one

In [116]:
# get the number of runners grouped by their nationality
queryString = """
SELECT ?country ?name COUNT(DISTINCT ?runner) AS ?runners
WHERE { 
    VALUES ?edition {wd:Q8558 wd:Q8567 wd:Q8577 wd:Q8613}.
    ?runner wdt:P106 wd:Q13382460;
            wdt:P27 ?country;
            wdt:P1344 ?edition.
    ?country sc:name ?name.
}
GROUP BY ?country ?name
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('runners', '2')]
[('country', 'http://www.wikidata.org/entity/Q717'), ('name', 'Venezuela'), ('runners', '3')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('name', 'Ukraine'), ('runners', '7')]
[('country', 'http://www.wikidata.org/entity/Q1030'), ('name', 'Namibia'), ('runners', '1')]
[('country', 'http://www.wikidata.org/entity/Q37'), ('name', 'Lithuania'), ('runners', '4')]
[('country', 'http://www.wikidata.org/entity/Q43'), ('name', 'Turkey'), ('runners', '6')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria'), ('runners', '1')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('name', 'Norway'), ('runners', '4')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia'), ('runners', '2')]
[('country', 'http://www.wikidata.org/entity/Q929'), ('name', 'Central African Republic'), ('runners', '1')]
[('country', 'http://www.wikidata.org/entity/Q225'), ('n

In [117]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    runners = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= runners
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_2
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_2.json
JSON object updated


## Task 7

In [6]:
# starting from Eliud Kipchoge I look at his properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q9124 ?pr ?obj.
    ?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('pr', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('pr', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('pr', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('pr', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('pr', 'http://www.wikidata.org/prop/direct/P1741'), ('name', 'GTAA ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('pr', 'http://www.wikidata.org/prop/direct/P19'

In [10]:
# starting from Eliud Kipchoge I look at his properties
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q9124 wdt:P1000 ?obj.
    ?obj sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q929585'), ('name', 'Marathon world record progression')]
1
